<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork865-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **SpaceX  Falcon 9 first stage Landing Prediction**


# Lab 1: Collecting the data


Estimated time needed: **45** minutes


In this capstone, we will predict if the Falcon 9 first stage will land successfully. SpaceX advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars; other providers cost upward of 165 million dollars each, much of the savings is because SpaceX can reuse the first stage. Therefore if we can determine if the first stage will land, we can determine the cost of a launch. This information can be used if an alternate company wants to bid against SpaceX for a rocket launch. In this lab, you will collect and make sure the data is in the correct format from an API. The following is an example of a successful and launch.


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/lab_v2/images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/lab_v2/images/crash.gif)


Most unsuccessful landings are planned. Space X performs a controlled landing in the oceans. 


## Objectives


In this lab, you will make a get request to the SpaceX API. You will also do some basic data wrangling and formating. 

- Request to the SpaceX API
- Clean the requested data


----


## Import Libraries and Define Auxiliary Functions


We will import the following libraries into the lab


In [1]:
# Requests allows us to make HTTP requests which we will use to get data from an API
import requests
# Pandas is a software library written for the Python programming language for data manipulation and analysis.
import pandas as pd
# NumPy is a library for the Python programming language, adding support for large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays
import numpy as np
# Datetime is a library that allows us to represent dates
import datetime

# Setting this option will print all collumns of a dataframe
pd.set_option('display.max_columns', None)
# Setting this option will print all of the data in a feature
pd.set_option('display.max_colwidth', None)

Below we will define a series of helper functions that will help us use the API to extract information using identification numbers in the launch data.

From the <code>rocket</code> column we would like to learn the booster name.


In [2]:
# Takes the dataset and uses the rocket column to call the API and append the data to the list
def getBoosterVersion(data):
    for x in data['rocket']:
       if x:
        response = requests.get("https://api.spacexdata.com/v4/rockets/"+str(x)).json()
        BoosterVersion.append(response['name'])

From the <code>launchpad</code> we would like to know the name of the launch site being used, the logitude, and the latitude.


In [3]:
# Takes the dataset and uses the launchpad column to call the API and append the data to the list
def getLaunchSite(data):
    for x in data['launchpad']:
       if x:
         response = requests.get("https://api.spacexdata.com/v4/launchpads/"+str(x)).json()
         Longitude.append(response['longitude'])
         Latitude.append(response['latitude'])
         LaunchSite.append(response['name'])

From the <code>payload</code> we would like to learn the mass of the payload and the orbit that it is going to.


In [46]:
def getPayloadData(data):
    for loads in data['payloads']:
        if loads:
            # Assuming there might be multiple payloads in a launch, loop through them
            for load in loads:
                response = requests.get("https://api.spacexdata.com/v4/payloads/"+str(load)).json()
                print("Response:", response)  # Add this line to check the response
                PayloadMass.append(response['mass_kg'])
                Orbit.append(response['orbit'])


From <code>cores</code> we would like to learn the outcome of the landing, the type of the landing, number of flights with that core, whether gridfins were used, wheter the core is reused, wheter legs were used, the landing pad used, the block of the core which is a number used to seperate version of cores, the number of times this specific core has been reused, and the serial of the core.


In [49]:
def getCoreData(data):
    for cores_list in data['cores']:
        for core in cores_list:
            if core['core'] is not None:
                response = requests.get("https://api.spacexdata.com/v4/cores/"+str(core['core'])).json()
                print("Response:", response)  # Add this line to check the response
                Block.append(response['block'])
                ReusedCount.append(response['reuse_count'])
                Serial.append(response['serial'])
            else:
                Block.append(None)
                ReusedCount.append(None)
                Serial.append(None)
            Outcome.append(str(core['landing_success'])+' '+str(core['landing_type']))
            Flights.append(core['flight'])
            GridFins.append(core['gridfins'])
            Reused.append(core['reused'])
            Legs.append(core['legs'])
            LandingPad.append(core['landpad'])


Now let's start requesting rocket launch data from SpaceX API with the following URL:


In [6]:
spacex_url="https://api.spacexdata.com/v4/launches/past"

In [7]:
response = requests.get(spacex_url)

Check the content of the response


You should see the response contains massive information about SpaceX launches. Next, let's try to discover some more relevant information for this project.


### Task 1: Request and parse the SpaceX launch data using the GET request


To make the requested JSON results more consistent, we will use the following static response object for this project:


In [9]:
static_json_url='https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/API_call_spacex_api.json'

We should see that the request was successfull with the 200 status response code


In [10]:
response.status_code

200

Now we decode the response content as a Json using <code>.json()</code> and turn it into a Pandas dataframe using <code>.json_normalize()</code>


In [28]:
# Use json_normalize meethod to convert the json result into a dataframe

# Make a GET request
response = requests.get(static_json_url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Decode the response content as JSON
    api_data = response.json()

    # Use json_normalize to convert the JSON data into a DataFrame
    launch_data = pd.json_normalize(api_data)

    # Display the first 5 rows of the DataFrame
    print(launch_data.head())
else:
    print(f"Request failed with status code: {response.status_code}")


       static_fire_date_utc  static_fire_date_unix    tbd    net  window  \
0  2006-03-17T00:00:00.000Z           1.142554e+09  False  False     0.0   
1                      None                    NaN  False  False     0.0   
2                      None                    NaN  False  False     0.0   
3  2008-09-20T00:00:00.000Z           1.221869e+09  False  False     0.0   
4                      None                    NaN  False  False     0.0   

                     rocket  success  \
0  5e9d0d95eda69955f709d1eb    False   
1  5e9d0d95eda69955f709d1eb    False   
2  5e9d0d95eda69955f709d1eb    False   
3  5e9d0d95eda69955f709d1eb     True   
4  5e9d0d95eda69955f709d1eb     True   

                                                                                                                                                                                details  \
0                                                                                                                  

Using the dataframe <code>data</code> print the first 5 rows


In [12]:
# Get the head of the dataframe


You will notice that a lot of the data are IDs. For example the rocket column has no information about the rocket just an identification number.

We will now use the API again to get information about the launches using the IDs given for each launch. Specifically we will be using columns <code>rocket</code>, <code>payloads</code>, <code>launchpad</code>, and <code>cores</code>.


In [30]:
# Let's take a subset of our dataframe keeping only the features we want and the flight number, and date_utc.
launch_data_subset = launch_data[['rocket', 'payloads', 'launchpad', 'cores', 'flight_number', 'date_utc']]

# We will remove rows with multiple cores because those are Falcon rockets with 2 extra rocket boosters and rows that have multiple payloads in a single rocket.
launch_data_subset = launch_data_subset[launch_data_subset['cores'].apply(len) == 1]
launch_data_subset = launch_data_subset[launch_data_subset['payloads'].apply(len) == 1]

# Since payloads and cores are lists of size 1, we will also extract the single value in the list and replace the feature.
launch_data_subset['cores'] = launch_data_subset['cores'].map(lambda x: x[0])
launch_data_subset['payloads'] = launch_data_subset['payloads'].map(lambda x: x[0])

# We also want to convert the date_utc to a datetime datatype and then extract the date leaving the time
launch_data_subset['date'] = pd.to_datetime(launch_data_subset['date_utc']).dt.date

# Using the date we will restrict the dates of the launches
launch_data_subset = launch_data_subset[launch_data_subset['date'] <= datetime.date(2020, 11, 13)]

* From the <code>rocket</code> we would like to learn the booster name

* From the <code>payload</code> we would like to learn the mass of the payload and the orbit that it is going to

* From the <code>launchpad</code> we would like to know the name of the launch site being used, the longitude, and the latitude.

* From <code>cores</code> we would like to learn the outcome of the landing, the type of the landing, number of flights with that core, whether gridfins were used, whether the core is reused, whether legs were used, the landing pad used, the block of the core which is a number used to seperate version of cores, the number of times this specific core has been reused, and the serial of the core.

The data from these requests will be stored in lists and will be used to create a new dataframe.


In [31]:
#Global variables 
BoosterVersion = []
PayloadMass = []
Orbit = []
LaunchSite = []
Outcome = []
Flights = []
GridFins = []
Reused = []
Legs = []
LandingPad = []
Block = []
ReusedCount = []
Serial = []
Longitude = []
Latitude = []

These functions will apply the outputs globally to the above variables. Let's take a looks at <code>BoosterVersion</code> variable. Before we apply  <code>getBoosterVersion</code> the list is empty:


In [32]:
BoosterVersion

[]

Now, let's apply <code> getBoosterVersion</code> function method to get the booster version


In [35]:
# Call getBoosterVersion
getBoosterVersion(launch_data)

the list has now been update 


In [36]:
BoosterVersion[0:5]

['Falcon 1', 'Falcon 1', 'Falcon 1', 'Falcon 1', 'Falcon 1']

we can apply the rest of the  functions here:


In [43]:
# Call getLaunchSite
getLaunchSite(launch_data)

In [47]:
# Call getPayloadData
getPayloadData(launch_data)

Response: {'dragon': {'capsule': None, 'mass_returned_kg': None, 'mass_returned_lbs': None, 'flight_time_sec': None, 'manifest': None, 'water_landing': None, 'land_landing': None}, 'name': 'FalconSAT-2', 'type': 'Satellite', 'reused': False, 'launch': '5eb87cd9ffd86e000604b32a', 'customers': ['DARPA'], 'norad_ids': [], 'nationalities': ['United States'], 'manufacturers': ['SSTL'], 'mass_kg': 20, 'mass_lbs': 43, 'orbit': 'LEO', 'reference_system': 'geocentric', 'regime': 'low-earth', 'longitude': None, 'semi_major_axis_km': None, 'eccentricity': None, 'periapsis_km': 400, 'apoapsis_km': 500, 'inclination_deg': 39, 'period_min': None, 'lifespan_years': None, 'epoch': None, 'mean_motion': None, 'raan': None, 'arg_of_pericenter': None, 'mean_anomaly': None, 'id': '5eb0e4b5b6c3bb0006eeb1e1'}
Response: {'dragon': {'capsule': None, 'mass_returned_kg': None, 'mass_returned_lbs': None, 'flight_time_sec': None, 'manifest': None, 'water_landing': None, 'land_landing': None}, 'name': 'DemoSAT', 't

In [50]:
# Call getCoreData
getCoreData(launch_data)

Response: {'block': None, 'reuse_count': 0, 'rtls_attempts': 0, 'rtls_landings': 0, 'asds_attempts': 0, 'asds_landings': 0, 'last_update': 'Engine failure at T+33 seconds resulted in loss of vehicle', 'launches': ['5eb87cd9ffd86e000604b32a'], 'serial': 'Merlin1A', 'status': 'lost', 'id': '5e9e289df35918033d3b2623'}
Response: {'block': None, 'reuse_count': 0, 'rtls_attempts': 0, 'rtls_landings': 0, 'asds_attempts': 0, 'asds_landings': 0, 'last_update': 'Successful first-stage burn and transition to second stage, maximal altitude 289 km. Harmonic oscillation at T+5 minutes Premature engine shutdown at T+7 min 30 s. Failed to reach orbit.', 'launches': ['5eb87cdaffd86e000604b32b'], 'serial': 'Merlin2A', 'status': 'lost', 'id': '5e9e289ef35918416a3b2624'}
Response: {'block': None, 'reuse_count': 0, 'rtls_attempts': 0, 'rtls_landings': 0, 'asds_attempts': 0, 'asds_landings': 0, 'last_update': 'Residual stage-1 thrust led to collision between stage 1 and stage 2.', 'launches': ['5eb87cdbffd8

Finally lets construct our dataset using the data we have obtained. We we combine the columns into a dictionary.


In [60]:
launch_dict = {'FlightNumber': list(launch_data['flight_number']),
'Date': list(launch_data['date_utc']),
'BoosterVersion':BoosterVersion,
'PayloadMass':PayloadMass,
'Orbit':Orbit,
'LaunchSite':LaunchSite,
'Outcome':Outcome,
'Flights':Flights,
'GridFins':GridFins,
'Reused':Reused,
'Legs':Legs,
'LandingPad':LandingPad,
'Block':Block,
'ReusedCount':ReusedCount,
'Serial':Serial,
'Longitude': Longitude,
'Latitude': Latitude}


In [58]:
# Display the column names of the DataFrame
print(launch_data.columns)

Index(['static_fire_date_utc', 'static_fire_date_unix', 'tbd', 'net', 'window',
       'rocket', 'success', 'details', 'crew', 'ships', 'capsules', 'payloads',
       'launchpad', 'auto_update', 'failures', 'flight_number', 'name',
       'date_utc', 'date_unix', 'date_local', 'date_precision', 'upcoming',
       'cores', 'id', 'fairings.reused', 'fairings.recovery_attempt',
       'fairings.recovered', 'fairings.ships', 'links.patch.small',
       'links.patch.large', 'links.reddit.campaign', 'links.reddit.launch',
       'links.reddit.media', 'links.reddit.recovery', 'links.flickr.small',
       'links.flickr.original', 'links.presskit', 'links.webcast',
       'links.youtube_id', 'links.article', 'links.wikipedia', 'fairings'],
      dtype='object')


Then, we need to create a Pandas data frame from the dictionary launch_dict.


In [64]:
# Determine the maximum length among all lists in launch_dict
max_length = max(map(len, launch_dict.values()))

# Extend lists to the maximum length
for key in launch_dict:
    launch_dict[key].extend([None] * (max_length - len(launch_dict[key])))


In [65]:
# Create a data from launch_dict
launch_data_df = pd.DataFrame(launch_dict)

Show the summary of the dataframe


In [66]:
# Show the head of the dataframe
launch_data_df.head()

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
0,1.0,2006-03-24T22:30:00.000Z,Falcon 1,20.0,LEO,Kwajalein Atoll,None None,1.0,False,False,False,None,NaN,0.0,Merlin1A,167.743129,9.047721
1,2.0,2007-03-21T01:10:00.000Z,Falcon 1,NaN,LEO,Kwajalein Atoll,None None,1.0,False,False,False,None,NaN,0.0,Merlin2A,167.743129,9.047721
2,3.0,2008-08-03T03:34:00.000Z,Falcon 1,165.0,LEO,Kwajalein Atoll,None None,1.0,False,False,False,None,NaN,0.0,Merlin1C,167.743129,9.047721
3,4.0,2008-09-28T23:15:00.000Z,Falcon 1,200.0,LEO,Kwajalein Atoll,None None,1.0,False,False,False,None,NaN,0.0,Merlin2C,167.743129,9.047721
4,5.0,2009-07-13T03:35:00.000Z,Falcon 1,NaN,LEO,Kwajalein Atoll,None None,1.0,False,False,False,None,NaN,0.0,Merlin3C,167.743129,9.047721


### Task 2: Filter the dataframe to only include `Falcon 9` launches


Finally we will remove the Falcon 1 launches keeping only the Falcon 9 launches. Filter the data dataframe using the <code>BoosterVersion</code> column to only keep the Falcon 9 launches. Save the filtered data to a new dataframe called <code>data_falcon9</code>.


In [67]:
# Assuming the column name for BoosterVersion is 'BoosterVersion'
data_falcon9 = launch_data_df[launch_data_df['BoosterVersion'] == 'Falcon 9']

# Display the first few rows of the filtered DataFrame
print(data_falcon9.head())

   FlightNumber                      Date BoosterVersion  PayloadMass Orbit  \
5           6.0  2010-06-04T18:45:00.000Z       Falcon 9        525.0   LEO   
6           7.0  2010-12-08T15:43:00.000Z       Falcon 9        677.0   ISS   
7           8.0  2012-05-22T07:44:00.000Z       Falcon 9        500.0    PO   
8           9.0  2012-10-08T00:35:00.000Z       Falcon 9       3170.0   GTO   
9          10.0  2013-03-01T19:10:00.000Z       Falcon 9       3325.0   GTO   

     LaunchSite    Outcome  Flights GridFins Reused   Legs LandingPad  Block  \
5  CCSFS SLC 40  None None      1.0    False  False  False       None    1.0   
6  CCSFS SLC 40  None None      1.0    False  False  False       None    1.0   
7  CCSFS SLC 40  None None      1.0    False  False  False       None    1.0   
8  CCSFS SLC 40  None None      1.0    False  False  False       None    1.0   
9  CCSFS SLC 40  None None      1.0    False  False  False       None    1.0   

   ReusedCount Serial  Longitude   Latitude 

Now that we have removed some values we should reset the FlgihtNumber column


In [68]:
data_falcon9.loc[:,'FlightNumber'] = list(range(1, data_falcon9.shape[0]+1))
data_falcon9

/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
5,1,2010-06-04T18:45:00.000Z,Falcon 9,525.0,LEO,CCSFS SLC 40,None None,1.0,False,False,False,None,1.0,0.0,B0003,-80.577366,28.561857
6,2,2010-12-08T15:43:00.000Z,Falcon 9,677.0,ISS,CCSFS SLC 40,None None,1.0,False,False,False,None,1.0,0.0,B0004,-80.577366,28.561857
7,3,2012-05-22T07:44:00.000Z,Falcon 9,500.0,PO,CCSFS SLC 40,None None,1.0,False,False,False,None,1.0,0.0,B0005,-80.577366,28.561857
8,4,2012-10-08T00:35:00.000Z,Falcon 9,3170.0,GTO,CCSFS SLC 40,None None,1.0,False,False,False,None,1.0,0.0,B0006,-80.577366,28.561857
9,5,2013-03-01T19:10:00.000Z,Falcon 9,3325.0,GTO,CCSFS SLC 40,None None,1.0,False,False,False,None,1.0,0.0,B0007,-80.577366,28.561857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209,194,None,Falcon 9,15600.0,VLEO,KSC LC 39A,None,NaN,None,None,None,None,NaN,NaN,None,-80.603956,28.608058
210,195,None,Falcon 9,15600.0,VLEO,KSC LC 39A,None,NaN,None,None,None,None,NaN,NaN,None,-80.603956,28.608058
211,196,None,Falcon 9,9525.0,ISS,CCSFS SLC 40,None,NaN,None,None,None,None,NaN,NaN,None,-80.577366,28.561857
212,197,None,Falcon 9,15600.0,VLEO,CCSFS SLC 40,None,NaN,None,None,None,None,NaN,NaN,None,-80.577366,28.561857


## Data Wrangling


We can see below that some of the rows are missing values in our dataset.


In [69]:
data_falcon9.isnull().sum()

FlightNumber        0
Date               99
BoosterVersion      0
PayloadMass        12
Orbit               0
LaunchSite          0
Outcome            98
Flights            98
GridFins           98
Reused             98
Legs               98
LandingPad        129
Block              98
ReusedCount        98
Serial             98
Longitude           0
Latitude            0
dtype: int64

Before we can continue we must deal with these missing values. The <code>LandingPad</code> column will retain None values to represent when landing pads were not used.


### Task 3: Dealing with Missing Values


Calculate below the mean for the <code>PayloadMass</code> using the <code>.mean()</code>. Then use the mean and the <code>.replace()</code> function to replace `np.nan` values in the data with the mean you calculated.


In [70]:
# Calculate the mean value of PayloadMass column
payload_mass_mean = data_falcon9['PayloadMass'].mean()

# Replace the np.nan values with its mean value
data_falcon9['PayloadMass'].replace(np.nan, payload_mass_mean, inplace=True)

# Verify the changes
print(data_falcon9.isnull().sum())

FlightNumber        0
Date               99
BoosterVersion      0
PayloadMass         0
Orbit               0
LaunchSite          0
Outcome            98
Flights            98
GridFins           98
Reused             98
Legs               98
LandingPad        129
Block              98
ReusedCount        98
Serial             98
Longitude           0
Latitude            0
dtype: int64


/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/pandas/core/generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [73]:
# Count the number of missing values in the 'LandingPad' column
missing_values_count = data_falcon9['LandingPad'].isnull().sum()

# Display the result
print(f"Number of missing values in the 'LandingPad' column: {missing_values_count}")


Number of missing values in the 'LandingPad' column: 129


You should see the number of missing values of the <code>PayLoadMass</code> change to zero.


In [74]:
data_falcon9.to_csv('dataset_part_1.csv', index=False)

Now we should have no missing values in our dataset except for in <code>LandingPad</code>.


We can now export it to a <b>CSV</b> for the next section,but to make the answers consistent, in the next lab we will provide data in a pre-selected date range. 


<code>data_falcon9.to_csv('dataset_part_1.csv', index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/joseph-s-50398b136/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork865-2022-01-01">Joseph Santarcangelo</a> has a PhD in Electrical Engineering, his research focused on using machine learning, signal processing, and computer vision to determine how videos impact human cognition. Joseph has been working for IBM since he completed his PhD. 


## Change Log


|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2020-09-20|1.1|Joseph|get result each time you run|
|2020-09-20|1.1|Azim |Created Part 1 Lab using SpaceX API|
|2020-09-20|1.0|Joseph |Modified Multiple Areas|


Copyright © 2021 IBM Corporation. All rights reserved.
